# Cleaning both Men and Women

In [59]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import brier_score_loss
from xgboost import XGBClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import json
from datetime import datetime
from datetime import datetime
import re

In [60]:
def get_bracket_data(CATEGORY):
    if CATEGORY == "mens":
        games = """
1 Auburn vs. 16 Alabama State/St. Francis PA
8 Louisville vs. 9 Creighton
5 Michigan vs. 12 UC San Diego
4 Texas A&M vs. 13 Yale
6 Ole Miss vs. 11 SDSU/North Carolina
3 Iowa State vs. 14 Lipscomb
7 Marquette vs. 10 New Mexico
2 Michigan State vs. 15 Bryant
1 Florida vs. 16 Norfolk State
8 UConn vs. 9 Oklahoma
5 Memphis vs. 12 Colorado State
4 Maryland vs. 13 Grand Canyon
6 Missouri vs. 11 Drake
3 Texas Tech vs. 14 UNC Wilmington
7 Kansas vs. 10 Arkansas
2 St. John’s vs. 15 Omaha
1 Duke vs. 16 American/Mount St. Mary’s
8 Mississippi State vs. 9 Baylor
5 Oregon vs. 12 Liberty
4 Arizona vs. 13 Akron
6 BYU vs. 11 VCU
3 Wisconsin vs. 14 Montana
7 St. Mary’s vs. 10 Vanderbilt
2 Alabama vs. 15 Robert Morris
1 Houston vs. 16 SIUE
8 Gonzaga vs. 9 Georgia
5 Clemson vs. 12 McNeese
4 Purdue vs. 13 High Point
6 Illinois vs. 11 Texas/Xavier
3 Kentucky vs. 14 Troy
7 UCLA vs. 10 Utah State
2 Tennessee vs. 15 Wofford
        """

        matchups = games.split("\n")[1:-1]
        m = [m.split(" vs. ") for m in matchups]
        brck = [re.sub(r'^\d+\s+', '', team) for pair in m for team in pair]



        bmap = {
            "Ole Miss":"Mississippi",
            "Iowa State":"Iowa St",
            "Michigan State":"Michigan St",
            "Norfolk State":"Norfolk St",
            "UConn":"Connecticut",
            "Colorado State":"Colorado St",
            "St. John’s":"St John's",
            "Omaha":"NE Omaha",
            "Mississippi State":"Mississippi St",
            "St. Mary’s":"St Mary's CA",
            "McNeese":"McNeese St",
            "Utah State":"Utah St",
            "SDSU":"South Dakota",
            "Mount St. Mary’s":"Mt St Mary's",
            "St. Francis PA":"St Francis PA",
            "Alabama State":"Alabama St",
            "American":"American Univ"}
    else:
        games = """
No. 1 UCLA vs. No. 16 UC San Diego/Southern
No. 8 Richmond vs. No. 9 Georgia Tech
No. 4 Baylor vs. No. 13 Grand Canyon
No. 5 Ole Miss vs. No. 12 Ball State
No. 3 LSU vs. No. 14 San Diego State
No. 6 Florida State vs. No. 11 George Mason
No. 2 NC State vs. No. 15 Vermont
No. 7 Michigan State vs. No. 10 Harvard
No. 1 USC vs. No. 16 UNC Greensboro
No. 8 California vs. No. 9 Mississippi State
No. 4 Kentucky vs. No. 13 Liberty
No. 5 Kansas State vs. No. 12 Fairfield
No. 3 Oklahoma vs. No. 14 Florida Gulf Coast
No. 6 Iowa vs. No. 11 Murray State
No. 2 UConn vs. No. 15 Arkansas State
No. 7 Oklahoma State vs. No. 10 South Dakota State
No. 1 South Carolina vs. No. 16 Tennessee Tech
No. 8 Utah vs. No. 9 Indiana
No. 4 Maryland vs. No. 13 Norfolk State
No. 5 Alabama vs. No. 12 Green Bay
No. 3 North Carolina vs. No. 14 Oregon State
No. 6 West Virginia vs. No. 11 Columbia/Washington
No. 2 Duke vs. No. 15 Lehigh
No. 7 Vanderbilt vs. No. 10 Oregon
No. 1 Texas vs. No. 16 High Point/William & Mary
No. 8 Illinois vs. No. 9 Creighton
No. 4 Ohio State vs. No. 13 Montana State
No. 5 Tennessee vs. No. 12 South Florida
No. 3 Notre Dame vs. No. 14 Stephen F. Austin
No. 6 Michigan vs. No. 11 Iowa State/Princeton
No. 2 TCU vs. No. 15 Fairleigh Dickinson
No. 7 Louisville vs. No. 10 Nebraska
        """

        matchups = games.split("\n")[1:-1]
        m = [m.split(" vs. ") for m in matchups]
        brck = [re.sub(r'^(No\.\s*)?\d+\s+', '', team).strip() for pair in m for team in pair]

        bmap = {
            "Ole Miss":"Mississippi",
            "Ball State":"Ball St",
            "San Diego State":"San Diego St",
            "Florida State":"Florida St",
            "Michigan State":"Michigan St",
            "Mississippi State":"Mississippi St",
            "Kansas State":"Kansas St",
            "Florida Gulf Coast":"FGCU",
            "Murray State":"Murray St",
            "UConn":"Connecticut",
            "Arkansas State":"Arkansas St",
            "Oklahoma State":"Oklahoma St",
            "South Dakota State":"South Dakota",
            "Norfolk State":"Norfolk St",
            "Green Bay":"WI Green Bay",
            "Oregon State":"Oregon St",
            "Ohio State":"Ohio St",
            "Montana State":"Montana St",
            "Stephen F. Austin":"SF Austin",
            "Fairleigh Dickinson":"F Dickinson",
            "Iowa State":"Iowa St",
            "Southern":"Southern Univ"
        }

    return bmap, brck

In [61]:
def data_cleaning(season, tournament, conferences):
    all_matches = pd.concat([season, tournament], axis=0).sort_values(["Season", "DayNum"]).reset_index(drop=True)
    all_matches.tail()
    all_matches["LLoc"] = all_matches.WLoc
    all_matches = all_matches.replace({"LLoc":{"H":"A", "A":"H", "N":"N"}})
    all_matches.head()
    cols = ["Season", "first_id", "second_id"]
    all_matches["first_id"] = all_matches[['WTeamID','LTeamID']].min(axis=1)
    all_matches["second_id"] = all_matches[['WTeamID','LTeamID']].max(axis=1)
    all_matches["prob"] = 0
    all_matches.loc[all_matches.first_id == all_matches.WTeamID, "prob"] = 1
    all_matches["game_id"] = all_matches[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    all_matches.head()
    adf = all_matches.drop(columns=["first_id", "second_id"])
    winning_cols = [c for c in adf.columns if c.startswith("W")]
    losing_cols = [c for c in adf.columns if c.startswith("L")]
    neutral_cols = [c for c in adf.columns if not(c.startswith("W") or c.startswith("L"))]

    #Figure out location for losing team

    df_w = adf[neutral_cols+winning_cols+["LScore"]].copy()
    df_l = adf[neutral_cols+losing_cols+["WScore"]].copy()

    df_w = df_w.rename(columns=lambda x:x[1:] if x.startswith("W") else x)
    df_l = df_l.rename(columns=lambda x:x[1:] if x.startswith("L") else x)
    df_l = df_l.rename(columns={"WScore":"points_allowed"})
    df_w = df_w.rename(columns={"LScore":"points_allowed"})

    df_w["result"] = 1
    df_l["result"] = 0

    df = pd.concat([df_w, df_l], ignore_index=True)
    df = df.sort_values(by=["Season", "TeamID", "DayNum"]).reset_index(drop=True)
    display(df.isna().sum())
    saved_df = df.copy()
    df = saved_df.copy()
    df = df.sort_values(by=["Season", "TeamID", "DayNum"]).reset_index(drop=True)
    stats = ["Score", "FGM", "FGA", "FGM3", "FGA3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF", "points_allowed"]
    cum_stats_cols = [f"cum_{s}" for s in stats]

    for stat in stats:
        df[f"cum_{stat}"] = df.groupby(["Season", "TeamID"])[stat].cumsum().shift(fill_value=0)

    df["games_won"] = df.groupby(["Season", "TeamID"])["result"].cumsum().shift(fill_value=0)

    df["games_played"] = df.groupby(["Season", "TeamID"]).cumcount()

    df["prev_TeamID"] = df["TeamID"].shift(1)
    for stat in [*cum_stats_cols, "games_played", "games_won"]:
        df.loc[df["TeamID"] != df["prev_TeamID"], stat] = 0

    df["games_lost"] = df["games_played"] - df["games_won"]

    df["win_percentage"] = df["games_won"]/df["games_played"]

    df = df.drop(columns=["prev_TeamID"])

    df = pd.merge(df, conferences, how="left", left_on=["Season", "TeamID"], right_on=["Season", "TeamID"])
    print(df.columns)
    df.head()
    cum_stats = df.drop(columns=stats)

    averages = ["cum_Score", "cum_OR", "cum_DR", "cum_Ast", "cum_TO", "cum_Stl", "cum_Blk", "cum_PF", "cum_points_allowed"]
    percentages = [("cum_FGM", "cum_FGA", "FG%"), ("cum_FGM3", "cum_FGA3", "FG3%"), ("cum_FTM", "cum_FTA", "FT%")]

    print(cum_stats[cum_stats.games_played == 0].shape)

    cum_stats = cum_stats[cum_stats.games_played != 0]

    for col in averages:
        colname = "avg_" + col[4:]
        cum_stats[colname] = cum_stats[col] / cum_stats["games_played"]

    for make, attempt, new_col in percentages:
        cum_stats[new_col] = cum_stats[make] / cum_stats[attempt]

    averages = cum_stats.drop(columns=cum_stats_cols)

    unimportant_cols = ["prob", "NumOT", "result", "Loc"]

    averages_to_merge = averages.drop(columns=unimportant_cols)
    safe = ['Season', 'DayNum', 'first_id', 'second_id', 'prob', 'game_id', 'NumOT']
    fdf = all_matches[safe]
    fdf.head()

    df_merged = pd.merge(fdf, averages_to_merge, how="left", left_on=["Season", "DayNum", "game_id", "first_id"], right_on=["Season", "DayNum", "game_id", "TeamID"])
    df_merged = df_merged.drop(columns=["TeamID"])
    for col in df_merged.columns:
        if col not in safe:
            df_merged[f"first_{col}"] = df_merged[col]
            df_merged = df_merged.drop(columns=[col])

    df_merged = pd.merge(df_merged, averages_to_merge, how="left", left_on=["Season", "DayNum", "game_id", "second_id"], right_on=["Season", "DayNum", "game_id", "TeamID"])
    df_merged.drop(columns=["TeamID"])
    for col in df_merged.columns:
        if col in safe or col.startswith("first_"):
            continue
        df_merged[f"second_{col}"] = df_merged[col]
        df_merged = df_merged.drop(columns=[col])

    nfg_df = df_merged.dropna()
    return nfg_df, averages

In [ ]:
def model(df, cat):
    not_needed = ["Season", "first_id", "second_TeamID", "second_id", "game_id", "NumOT", "DayNum"]

    df = df.drop(columns=not_needed)

    X = df.drop(columns=["prob"])
    y = df["prob"]

    # Identify categorical columns
    categorical_features = X.select_dtypes(include=['object', 'category']).columns

    # ColumnTransformer to apply OneHotEncoder only to categorical columns
    preprocessor = ColumnTransformer(
        transformers=[
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features)
        ],
        remainder="passthrough"  # Keep non-categorical columns as they are
    )

    param_grid = {
        'n_estimators': Integer(10, 1000),
        'max_depth': Integer(3, 25),
        'learning_rate': Real(0.01, 0.5, prior='log-uniform'),
        'subsample': Real(0.5, 1.0),
        'colsample_bytree': Real(0.5, 1.0)
    }

    classifier = XGBClassifier(device="cuda")

    grid_search = BayesSearchCV(classifier, param_grid, scoring="neg_brier_score", cv=5, verbose=3, n_iter=25)

    pipe = Pipeline([
        ("preprocessing", preprocessor),
        ("classifier", grid_search)  
    ])

    pipe.fit(X, y)

    print("Best set of hyperparameters: ", pipe.named_steps["classifier"].best_params_)
    print("Best score: ", -pipe.named_steps["classifier"].best_score_)

    with open(f"./model_params/{cat}_{datetime.isoformat(datetime.now())}.json", "w") as f:
        temp = pipe.named_steps["classifier"].best_params_
        temp["score"] = -pipe.named_steps["classifier"].best_score_
        json.dump(temp, f)

    return pipe

In [63]:
def get_data(CATEGORY):
    if CATEGORY == "mens":
        season = pd.read_csv("./data/MRegularSeasonDetailedResults.csv")
        tournament = pd.read_csv("./data/MNCAATourneyDetailedResults.csv")
        conferences = pd.read_csv("./data/MTeamConferences.csv")
    else:
        season = pd.read_csv("./data/WRegularSeasonDetailedResults.csv")
        tournament = pd.read_csv("./data/WNCAATourneyDetailedResults.csv")
        conferences = pd.read_csv("./data/WTeamConferences.csv")
    return season, tournament, conferences

In [64]:
def make_predictions(CATEGORY, averages, pipe):
    submission = pd.read_csv("./data/SampleSubmissionStage2.csv")
    teams = pd.DataFrame(submission["ID"].str.split("_").to_list())
    cols = ["Season", "first_team_id", "second_team_id"]
    teams.columns = cols
    teams = pd.concat([teams, submission], axis=1)
    for c in cols:
        teams[c] = teams[c].astype("int64")

    if CATEGORY == "mens":
        teams = teams[teams.first_team_id < 2000]
    else:
        teams = teams[teams.first_team_id > 2000]

    temp_avgs = averages.drop(columns=["NumOT", "prob", "Loc", "result"])
    data2025 = temp_avgs[temp_avgs.Season == 2025]
    last_info = data2025.loc[data2025.groupby(["TeamID"])["DayNum"].idxmax()]

    d = pd.merge(teams, last_info, how="left", left_on="first_team_id", right_on="TeamID", suffixes=("", "_first"))
    d=d.drop(columns=["Season_first", "DayNum", "TeamID", "game_id"])
    d.columns = [f"first_{col}" if col not in teams.columns else col for col in d.columns ]
    d.head()
    d = pd.merge(d, last_info, how="left", left_on="second_team_id", right_on="TeamID", suffixes=("", "_second"))
    d = d.drop(columns=["Season_second", "DayNum", "game_id", "TeamID"])
    d.columns = [f"second_{col}" if col in last_info.columns else col for col in d.columns ]
    d = d.drop(columns=["second_Season", "first_team_id", "second_team_id"])
    d.head()

    preds = pipe.predict_proba(d)[:,1]

    teams["Pred"] = preds

    submission = teams[["ID", "Pred"]]
    return submission, teams

In [65]:
def get_tm_id(tm_name, cpy_tms):
    cpy_tms = cpy_tms[["TeamName", "TeamID"]]
    return cpy_tms.loc[cpy_tms.TeamName == tm_name, "TeamID"].iloc[0]

def get_result(round, team_df, map, team_names):
    round = [
        [map.get(round[0], round[0]), 0], 
        [map.get(round[1], round[1]), 0]
    ]
    round[0][1] = get_tm_id(round[0][0], team_names)
    round[1][1] = get_tm_id(round[1][0], team_names)
    round.sort(key=lambda x:x[1])
    first = round[0][0]
    second = round[1][0]
    pred = team_df.loc[(team_df.first_team_name == first) & (team_df.second_team_name == second), "Pred"]
    # print(pred)
    pred = pred.iloc[0]
    print(first if pred > .5 else  second, "beat", second if pred > .5 else first, "with a prediction of", pred if pred > .5 else 1-pred)
    return first if pred > .5 else second


def team_prediction(bracket, df, map, team_names):
    # One round for play ins:
    next_round = []
    for tm in bracket:
        winner = tm
        if "/" in tm:
            winner = get_result(tm.split("/"), df, map, team_names)
        next_round.append(winner)
    bracket = next_round

    #Loop through since they are all 0 when %2
    while len(bracket) != 1:
        next_round = []
        while len(bracket) != 0:
            curr_round = []
            for i in range(2):
                curr_round.append(bracket.pop(0))
            winner = get_result(curr_round, df, map, team_names)
            next_round.append(winner)
        bracket = next_round

def print_bracket_results(teams, CATEGORY, brck, bmap):

    if CATEGORY == "mens":
        tms = pd.read_csv("./data/MTeams.csv")
    else:
        tms = pd.read_csv("./data/WTeams.csv")


    cpy_tms = tms.copy()
    tms = tms[["TeamID", "TeamName"]]

    teams = teams.merge(tms, how="left", left_on="first_team_id", right_on="TeamID")
    teams["first_team_name"] = teams.TeamName
    teams = teams.drop(columns=["TeamName", "TeamID"])

    teams = teams.merge(tms, how="left", left_on="second_team_id", right_on="TeamID")
    teams["second_team_name"] = teams.TeamName
    teams = teams.drop(columns=["TeamName", "TeamID"])

    print(f"{CATEGORY} BRACKET PREDICTIONS")
    team_prediction(brck, teams, bmap, cpy_tms)

In [68]:
# Running everything
results = pd.DataFrame()
for cat in ["mens", "womens"]:
    print("Cleaning data...")
    season, tournament, conferences = get_data(cat)
    df, averages = data_cleaning(season, tournament, conferences)
    print("Modeling...")
    pipe = model(df, cat)
    print("Making Predictions...")
    bmap, brck = get_bracket_data(cat)
    submission, teams = make_predictions(cat, averages, pipe)
    submission.to_csv(f"./individual_submissions/{cat}_{datetime.isoformat(datetime.now())}", index=False)
    print_bracket_results(teams, cat, brck, bmap)
    results = pd.concat([results, submission], axis=0)

Cleaning data...


Season            0
DayNum            0
NumOT             0
prob              0
game_id           0
TeamID            0
Score             0
Loc               0
FGM               0
FGA               0
FGM3              0
FGA3              0
FTM               0
FTA               0
OR                0
DR                0
Ast               0
TO                0
Stl               0
Blk               0
PF                0
points_allowed    0
result            0
dtype: int64

Index(['Season', 'DayNum', 'NumOT', 'prob', 'game_id', 'TeamID', 'Score',
       'Loc', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast',
       'TO', 'Stl', 'Blk', 'PF', 'points_allowed', 'result', 'cum_Score',
       'cum_FGM', 'cum_FGA', 'cum_FGM3', 'cum_FGA3', 'cum_FTM', 'cum_FTA',
       'cum_OR', 'cum_DR', 'cum_Ast', 'cum_TO', 'cum_Stl', 'cum_Blk', 'cum_PF',
       'cum_points_allowed', 'games_won', 'games_played', 'games_lost',
       'win_percentage', 'ConfAbbrev'],
      dtype='object')
(7981, 28)
Modeling...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END colsample_bytree=0.64918103501114, learning_rate=0.14481351804342968, max_depth=13, n_estimators=662, subsample=0.6746805074986886;, score=-0.237 total time=  28.7s
[CV 2/5] END colsample_bytree=0.64918103501114, learning_rate=0.14481351804342968, max_depth=13, n_estimators=662, subsample=0.6746805074986886;, score=-0.240 total time=  23.6s


KeyboardInterrupt: 